# Combining LCEL Chains

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 007-main-ops-lcel-chain.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **007-main-ops-lcel-chain**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [5]:
#!pip install langchain

In [6]:
#!pip install langchain lanchain-community

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [7]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [8]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo")

## Coercion: a chain inside another chain
* Remember: almost any component in LangChain (prompts, models, output parsers, etc) can be used as a Runnable.
* **Runnables can be chained together using the pipe operator `|`. The resulting chains of runnables are also runnables themselves**.

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")


model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

chain = prompt | model | StrOutputParser()

In [11]:
chain.invoke("Chamberlain")

'Neville Chamberlain, as Prime Minister of Britain, pursued a policy of appeasement towards Nazi Germany in the lead-up to World War II.'

#### Coercion: combine a chain with other Runnables to create a new chain.
* See how in the `composed_chain` we are including the previous `chain`:

In [12]:
from langchain_core.output_parsers import StrOutputParser

historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [13]:
composed_chain.invoke({"politician": "Lincoln"})

"Most historians and the general public view Abraham Lincoln as having a profoundly positive impact on humanity. Here's why:\n\n*   **Preservation of the Union:** Lincoln's leadership during the Civil War was crucial in preventing the permanent fracturing of the United States. He believed in the importance of a unified nation and fought to maintain it.\n*   **Abolition of Slavery:** While his initial focus was on preserving the Union, Lincoln's views on slavery evolved. The Emancipation Proclamation, though initially a military strategy, ultimately paved the way for the complete abolition of slavery in the United States. This was a monumental step forward for human rights and equality.\n*   **Moral Leadership:** Lincoln's speeches and writings often emphasized the importance of equality, justice, and the rights of all people. He provided moral leadership during a time of great division and uncertainty.\n*   **The 13th Amendment:** Lincoln played a significant role in the passage of the

In [14]:
composed_chain.invoke({"politician": "Attila"})

'Attila the Hun was definitely not positive for humanity. His military campaigns across Europe in the 5th century were characterized by widespread violence, destruction, and immense suffering.\n\nHere\'s why he\'s considered a destructive figure:\n\n*   **Brutal Warfare:** The Huns, under Attila\'s leadership, were known for their ruthless tactics, including pillaging, looting, and massacring entire populations.\n*   **Widespread Devastation:** Attila\'s campaigns led to the destruction of countless settlements and infrastructure, disrupting economies and causing widespread famine.\n*   **Forced Displacement:** The Hunnic invasions forced many people to flee their homes, creating refugee crises and contributing to the instability of the late Roman Empire.\n*   **Fear and Terror:** Attila\'s reputation as the "Scourge of God" instilled fear and terror throughout Europe, leading to social and political upheaval.\n\nWhile some historians argue that the Hunnic invasions indirectly contribu

## Another example: a chain inside another chain

In [15]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"politician": "Miterrand", "language": "French"})

"L'Europe."

## Fallback for Chains
* When working with language models, you may often encounter issues from the underlying APIs, whether these be rate limiting or downtime. Therefore, as you go to move your LLM applications into production it becomes more and more important to safeguard against these. That's why LangChain introduced the concept of fallbacks.
* A fallback is an alternative plan that may be used in an emergency.
* Fallbacks can be applied not only on the LLM level but on the whole runnable level. This is important because often times different models require different prompts. So if your call to OpenAI fails, you don't just want to send the same prompt to Anthropic - you probably want to use a different prompt template and send a different version there.
* We can create fallbacks for LCEL chains. Here we do that with two different models: ChatOpenAI (with a bad model name to easily create a chain that will error) and then normal OpenAI (which does not use a chat model). Because OpenAI is NOT a chat model, you likely want a different prompt.

In [16]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a funny assistant who always includes a joke in your response",
        ),
        ("human", "Who is the best {sport} player worldwide?"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

bad_chain = chat_prompt | chat_model | StrOutputParser()

In [17]:
# Now lets create a chain with the normal OpenAI model
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

prompt_template = """Instructions: You're a funny assistant who always includes a joke in your response.

Question: Who is the best {sport} player worldwide?"""

prompt = PromptTemplate.from_template(prompt_template)

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

good_chain = prompt | llm

In [18]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])

chain.invoke({"sport": "soccer"})

'Ah, the age-old question! Declaring the "best" soccer player is like trying to pick the best flavor of ice cream – everyone has their own favorite!\n\nCurrently, it\'s a hot debate between Lionel Messi and Cristiano Ronaldo, with some rising stars like Kylian Mbappé also entering the conversation. Each player has unique strengths and accomplishments that make them top contenders.\n\nBut, if you ask me, the best soccer player is always the one who brings the most snacks to the game!\n\nWhy did the soccer ball quit the team?\nBecause it was tired of being kicked around!'

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 007-main-ops-lcel-chain.py
* In terminal, make sure you are in the directory of the file and run:
    * python 007-main-ops-lcel-chain.py